In [1]:
# HL23PYR Stage 1: Healthy Neuron Simulation and Validation

# ✅ Goal: Simulate the healthy HL23PYR neuron using NetPyNE/NEURON,
# run current injection protocol (170–310 pA), extract features with EFEL,
# and compare to AllenDB human data (Cell ID: 531526539).

In [ ]:
# --- Imports and Setup ---

from netpyne import specs
from netpyne import sim
from neuron import h
import matplotlib.pyplot as plt
import numpy as np
import efel
import os

In [ ]:
!nrnivmodl

In [4]:
# # --- Load compiled mod mechanisms ---
# dll_path = './arm64/libnrnmech.dylib'
# if os.path.isfile(dll_path):
#     h.nrn_load_dll(dll_path)
# else:
#     raise RuntimeError(f"Missing NEURON mechanisms at {dll_path}. Run `nrnivmodl`.")

In [ ]:
# --- Define NetParams and Load HL23PYR Cell ---
netParams = specs.NetParams()
cellName = 'HL23PYR'
cellRule = netParams.importCellParams(
    label=cellName,
    conds={'cellType': cellName, 'cellModel': 'HH_full'},
    fileName='cellwrapper.py',
    cellName='loadCell_' + cellName,
    cellArgs={'cellName': cellName},
    cellInstance=True,
    somaAtOrigin=False)

In [ ]:
# --- Define SimConfig ---
simConfig = specs.SimConfig()
simConfig.duration = 500  # ms
simConfig.dt = 0.025
simConfig.verbose = False
simConfig.recordTraces = {'V_soma': {'sec': 'soma', 'loc': 0.5, 'var': 'v'}}
simConfig.recordStep = 0.1
simConfig.filename = 'HL23PYR_stage1'

In [ ]:
# --- Run multiple current injections (sweeps 45–52 = 170–310 pA) ---
step_amps = np.arange(170, 311, 20)  # pA
responses = []

for amp in step_amps:
    stim = h.IClamp(sim.net.cells[0].secDict['soma'][0])
    stim.delay = 100
    stim.dur = 300
    stim.amp = amp / 1000  # convert pA to nA
    sim.runSim()
    sim.gatherData()
    V = sim.simData['V_soma']
    t = sim.simData['t']
    responses.append({'amp': amp, 't': t.copy(), 'V': V.copy()})

In [ ]:
# --- Plot Voltage Responses ---
plt.figure(figsize=(10, 6))
for r in responses:
    plt.plot(r['t'], r['V'], label=f"{r['amp']} pA")
plt.xlabel('Time (ms)')
plt.ylabel('Vm (mV)')
plt.title('HL23PYR Voltage Responses to Step Currents')
plt.legend()
plt.show()

In [ ]:
# --- Prepare EFEL Input for Feature Extraction ---
ef_features = []
for r in responses:
    ef_dict = {
        'stim_start': [100],
        'stim_end': [400],
        'T': r['t'],
        'V': r['V'],
        'stimulus_current': [r['amp'] / 1000],
    }
    ef_features.append(ef_dict)

In [ ]:
# --- Extract EFEL Features ---
feature_names = [
    'peak_voltage', 'AP_amplitude', 'AP_width',
    'adaptation_index', 'sag_ratio2', 'voltage_base',
    'mean_frequency', 'time_to_first_spike'
]
results = efel.getMeanFeatureValues(ef_features, feature_names)

In [ ]:
# --- Print Extracted Features ---
for i, r in enumerate(results):
    print(f"\nSweep {step_amps[i]} pA:")
    for feat, val in r.items():
        print(f"  {feat}: {val}")